In [1]:
import what.utils.logger as log

logger = log.get_logger(__name__)

In [2]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
pretrained_ssd = "models/mobilenet-v1-ssd-mp-0_675.pth"
checkpoint_folder = "models/"

# Load Training Dataset

In [4]:
batch_size = 32
num_workers = 0

In [5]:
import fiftyone as fo
import fiftyone.zoo as foz
from torch.utils.data import DataLoader

from what.models.detection.ssd.datasets.fiftyone import FiftyOneDataset

In [6]:
from what.models.detection.ssd.ssd.preprocessing import TrainAugmentation, TestTransform
from what.models.detection.ssd.ssd.config import mobilenetv1_ssd_config
from what.models.detection.ssd.ssd.ssd import MatchPrior

In [7]:
# Load MobileNetSSD configuration
config              = mobilenetv1_ssd_config
train_transform     = TrainAugmentation(config.image_size, config.image_mean, config.image_std)
target_transform    = MatchPrior(config.priors, config.center_variance, config.size_variance, 0.5)
test_transform      = TestTransform(config.image_size, config.image_mean, config.image_std)

In [8]:
# Load the data from fiftyone
voc_2012_train = foz.load_zoo_dataset("voc-2012", split="train")

Split 'train' already downloaded
Loading 'voc-2012' split 'train'
 100% |███████████████| 5717/5717 [12.2s elapsed, 0s remaining, 373.4 samples/s]      
Dataset 'voc-2012-train' created


In [9]:
# Visualize the dataset
session = fo.launch_app(voc_2012_train)

In [10]:
# Convert to pytorch dataloader
train_dataset = FiftyOneDataset(voc_2012_train, 
                                foz.load_zoo_dataset_info("voc-2012").classes, 
                                transform=train_transform,
                                target_transform=target_transform)

# Load Training Dataset from Local Disk
# train_dataset = VOCDataset("examples/VOC2012", transform=train_transform,
#                              target_transform=target_transform)

train_loader  = DataLoader(train_dataset, batch_size,
                          num_workers=num_workers,
                          shuffle=True)

logger.info("Train dataset size: {}".format(len(train_dataset)))

[INFO] 2022-01-14 16:08:22,270 __main__: Train dataset size: 5717


# Load Validation Dataset

In [11]:
# Load Validation Dataset from FiftyOne (use voc-2007 train as validation here)
val_dataset  = FiftyOneDataset(foz.load_zoo_dataset("voc-2007", split="train"), 
                                        foz.load_zoo_dataset_info("voc-2007").classes,
                                        transform=test_transform,
                                        target_transform=target_transform)

# Load Validation Dataset from Local Disk
# val_dataset = VOCDataset("VOC2007/", transform=test_transform,
#                              target_transform=target_transform, is_test=True)

val_loader = DataLoader(val_dataset, batch_size,
                        num_workers=num_workers,
                        shuffle=False)

logger.info("validation dataset size: {}".format(len(val_dataset)))

Split 'train' already downloaded
Loading 'voc-2007' split 'train'
 100% |███████████████| 2501/2501 [6.6s elapsed, 0s remaining, 348.8 samples/s]       
Dataset 'voc-2007-train' created


[INFO] 2022-01-14 16:08:29,287 __main__: validation dataset size: 2501


# Train Mobilenet V1 SSD

In [12]:
from what.models.detection.ssd.mobilenet_v1_ssd import MobileNetV1SSD

In [13]:
# Create SSD network and load pretrained base net.
model = MobileNetV1SSD(is_test=False)

model.train(train_loader, val_loader, device=device, num_epochs=1, debug_steps=10, validation_epochs=1,
           pretrained_ssd = pretrained_ssd, checkpoint_folder = checkpoint_folder)

[INFO] 2022-01-14 16:08:29,387 what.models.detection.ssd.mobilenet_v1_ssd: Init from pretrained ssd models/mobilenet-v1-ssd-mp-0_675.pth
[INFO] 2022-01-14 16:08:29,463 what.models.detection.ssd.mobilenet_v1_ssd: Start training using CosineAnnealingLR scheduler.
[INFO] 2022-01-14 16:10:14,549 what.models.detection.ssd.mobilenet_v1_ssd: Epoch: 0, Step: 10, Average Loss: 19.7178, Average Regression Loss 3.9547, Average Classification Loss: 15.7630
[INFO] 2022-01-14 16:11:50,808 what.models.detection.ssd.mobilenet_v1_ssd: Epoch: 0, Step: 20, Average Loss: 14.1856, Average Regression Loss 3.1112, Average Classification Loss: 11.0743
[INFO] 2022-01-14 16:13:27,665 what.models.detection.ssd.mobilenet_v1_ssd: Epoch: 0, Step: 30, Average Loss: 11.7552, Average Regression Loss 2.9490, Average Classification Loss: 8.8062
[INFO] 2022-01-14 16:15:05,689 what.models.detection.ssd.mobilenet_v1_ssd: Epoch: 0, Step: 40, Average Loss: 9.8950, Average Regression Loss 2.8774, Average Classification Loss: 